<a href="https://colab.research.google.com/github/Deepti2412/Codes/blob/master/Text_Generation_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('moby_dick_four_chapters.txt','r') as f:
  r=f.read()
print(r)

In [4]:
r[:50]

'Call me Ishmael.  Some years ago--never mind how l'

In [5]:
import spacy

In [6]:
nlp=spacy.load('en',diable=['parser','tagger','ner'])

In [7]:
nlp.max_length=1198623

In [8]:
def separate_punct(doc):
  return [token.text.lower() for token in nlp(doc) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [9]:
tokens=separate_punct(r)

In [10]:
len(tokens)

11338

Create Sequences of Tokens

In [11]:
train_len= 25 + 1
text_sequences=[]
for i in range(train_len,len(tokens)):
  sequences=tokens[i-train_len:i]
  text_sequences.append(sequences)


In [12]:
" ".join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [13]:
" ".join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [14]:
len(text_sequences)

11312

KERAS

In [15]:
from keras_preprocessing.text import Tokenizer

In [16]:
tokenizer=Tokenizer()


In [17]:
tokenizer.fit_on_texts(text_sequences)

In [18]:
sequences=tokenizer.texts_to_sequences(text_sequences)

In [ ]:
sequences[2]

In [ ]:
tokenizer.index_word

In [ ]:
for i in sequences[0]:
  print(f'{i} {tokenizer.index_word[i]}')

In [22]:
vocabulary_size=len(tokenizer.word_counts)

In [24]:
vocabulary_size

2717

In [25]:
import numpy as np

In [26]:
sequences=np.array(sequences)

In [27]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

Creating an LSTM based model

In [28]:
from keras.layers import Dense,LSTM,Embedding
from keras.models import Sequential

Using TensorFlow backend.


In [30]:
def create_model(vocabulary_size,seq_len):
  model=Sequential()
  model.add(Embedding(vocabulary_size,25,input_length=seq_len))
  model.add(LSTM(150,return_sequences=True))
  model.add(LSTM(150))
  model.add(Dense(150,activation='relu'))
  model.add(Dense(vocabulary_size, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
  model.summary()
    
  return model

Train / Test Split

In [31]:
from keras.utils import to_categorical

In [32]:
X=sequences[:,:-1]
y=sequences[:,-1]

In [ ]:
y

In [ ]:
X

In [ ]:
sequences

In [33]:
y=to_categorical(y,num_classes=vocabulary_size+ 1)

In [34]:
seq_len = X.shape[1]

In [35]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 150)           105600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_2 (Dense)              (None, 2718)              410418    
Total params: 787,218
Trainable params: 787,218
Non-trainable params: 0
_________________________________________________________________


In [36]:
from pickle import dump,load

In [37]:
# fit model
model.fit(X, y, batch_size=128, epochs=300,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/300
11312/11312 [==============================] - 25s 2ms/step - loss: 6.8471 - accuracy: 0.0463
Epoch 2/300
11312/11312 [==============================] - 24s 2ms/step - loss: 6.3854 - accuracy: 0.0510
Epoch 3/300
11312/11312 [==============================] - 24s 2ms/step - loss: 6.3274 - accuracy: 0.0522
Epoch 4/300
11312/11312 [==============================] - 24s 2ms/step - loss: 6.1828 - accuracy: 0.0529
Epoch 5/300
11312/11312 [==============================] - 24s 2ms/step - loss: 6.1014 - accuracy: 0.0542
Epoch 6/300
11312/11312 [==============================] - 24s 2ms/step - loss: 5.9784 - accuracy: 0.0612
Epoch 7/300
11312/11312 [==============================] - 24s 2ms/step - loss: 5.8620 - accuracy: 0.0670
Epoch 8/300
11312/11312 [==============================] - 24s 2ms/step - loss: 5.7837 - accuracy: 0.0680
Epoch 9/300
11312/11312 [==============================] - 24s 2ms/step - loss: 5.7071 - accuracy: 0.0693
Epoch 10/300
11312/11312 [====================

In [38]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [39]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [40]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [42]:
random_seed_text = text_sequences[random_pick]

In [44]:
seed_text = ' '.join(random_seed_text)

In [45]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

"to be afraid of him better sleep with a sober cannibal than a drunken christian landlord said i tell him to stash his tomahawk there or pipe or whatever you call it tell him to stop smoking in short and i will turn in with him but i do n't"

Exploring Generated Sequence

In [52]:
for i,word in enumerate(r.split()):
    if word == 'stash':
        print(' '.join(r.split()[i-20:i+30]))
        print('\n')

be afraid of him. Better sleep with a sober cannibal than a drunken Christian. "Landlord," said I, "tell him to stash his tomahawk there, or pipe, or whatever you call it; tell him to stop smoking, in short, and I will turn in with him. But I don't fancy having


